In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import lognorm
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import lognorm
import statsmodels.api as sm
from sklearn.preprocessing import PowerTransformer

#  1
# !pip install datacompy
#  download it in the file where you are importing
import datacompy
import random
import scipy.stats
import math
from scipy import stats
from scipy.stats import ks_2samp
from keras.layers import Input,Dense
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#  2
# !pip install pyclustering
# download it in the file where you are importing

from sklearn.mixture import GaussianMixture
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import DBSCAN


from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix


ModuleNotFoundError: No module named 'datacompy'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
t=pd.read_csv(input('Enter the path of your dataset'))
# /content/train (5).csv


In [ ]:
print('No. of observations:',len(t))

In [ ]:
def preprocessing(train):
  train=train.reset_index()
  train.drop('index',axis=1,inplace=True)
  print(train.head(),'\n')
  print(train.info(),'\n')
  print('Columns are:',train.columns,'\n')
  print(train.describe(),'\n')

  print('enter the index column name: ')
  index_col=input()
  print('Enter categorical columns: ')
  categories=list(map(str,input().split()))
  numeric=[]
  train1=train.copy()
  for i in train1.columns:
    if i not in categories and i!=index_col:
      numeric.append(i)

  """
    CHECK FOR MISSING VALUES.
  IMPUTE DATA IF NEEDED"""
  def check_and_fill(train):
    print('Enter the missing threshold and the correlation threshold: ')
    miss_perc,corr_threshold=map(float,input().split())
    for col in train.columns:
      if(train[col].isnull().sum()==0):
        pass
      else:
        countMiss=train[col].isnull().sum()
        numOfObs=len(train)
        if(countMiss/numOfObs*100)>miss_perc and -(corr_threshold)<train[col].corr(train['target'])<corr_threshold:
          train=train.drop(col,axis=1)
        elif countMiss>0:
          imputeVal=input('Enter 1 for KNNImputer, 2 for Simple Imputer: ')
          if imputeVal=='1':
            impute=KNNImputer(n_neighbors=2)
          elif imputeVal=='2':
            impute=SimpleImputer(strategy='most_frequent')
          train=impute.fit_transform(train)
    return train

  train1=pd.DataFrame(check_and_fill(train1))
  #numeric and categories- lists which store
  # train1.columns
  if index_col in train1:
    train1.drop(index_col,axis=1,inplace=True) #dropping id column

  #check for duplicates,remove them if needed
  train1=train1.drop_duplicates()

  #SHOWING DISTRIBUTION BALANCE OF CATEGORICAL DATA
  def plot_categorical_balance(train,categories):
    train=train[categories]
    for col in categories:
      sns.countplot(x=col,data=train)
  plot_categorical_balance(train1,categories)

  #Scaling the data
  train2=train1.copy()
  train2=train2.drop(categories,axis=1)

  def scaling(train,scaler_type):
    if scaler_type==1:
      scaler=StandardScaler()
    elif scaler_type==2:
      scaler=RobustScaler()
    elif scaler_type==3:
      scaler=MinMaxScaler()
    scaled_train=scaler.fit_transform(train)
    return scaled_train

  def get_scaler_number():
    while True:
      try:
        scaler_type=int(input('enter 1 for Standard Scaler,2 for Robust Scaler, 3 for Min Max Scaler: '))
        l=[1,2,3]
        if scaler_type in l:
          return scaler_type
        else:
          print("Invalid input. Please enter from the given list: ",l)
      except ValueError:
        print("Invalid input. Please enter correctly. ")

  scaler_no=get_scaler_number()

  #train3 will store the numeric scaled data
  train3=pd.DataFrame(scaling(train2,scaler_no))

  #train2 will store the scaled data and the categorical data
  train2=pd.concat([train1[categories],train2],axis=1)

  #Applying one hot encoding to the categorical columns except the target column
  def one_hot_encoding(train,categories):
    new_train=pd.DataFrame()
    for col in categories:
      if col!='target':
        one_hot_data=pd.get_dummies(col,prefix=col,drop_first=True)
        train.drop(col,axis=1,inplace=True)
        new_train=pd.concat([new_train,one_hot_data],axis=1)
      if col=='target':
        new_train=pd.concat([new_train,train['target']],axis=1)
    return new_train

  #Stores the one hot encoded data only
  train4=one_hot_encoding(train2,categories)

  #Combining the one hot encoded data and the scaled data
  train2=pd.concat([train3,train4],axis=1)
  print('DATA after one-hot encoding: ',train2)

  ytrain=train1['target'] # or get it from train2
  #keep target column in one variable, train1 to get target values.
  print(train2.info())
  print(train2.columns)

  #Univariate analysis
  #Plotting Q-Q plots to check for normalization
  #Check the skewness of data columns
  import numpy as np
  from scipy.stats import lognorm
  import statsmodels.api as sm
  np.random.seed(1)
  # from scipy.stats import shapiro
  import random
  def plot_histograms(train,numeric):
    feature=(np.random.choice(numeric))
    plt.figure(figsize=(20,10))
    sns.distplot(train[feature])
    skew_val=train1.skew(axis=1)
    print(f"Skewness of {feature}: {skew_val}")
    #If plot is not along 45 degrees,dataset is not normally distributed, see visually
    sm.qqplot(train1,line='45')
    plt.show()
  plot_histograms(train2,train3.columns)

  print('Scatterplot')
  def plot_scatter(train3):
    a=np.random.choice(train1.columns)
    b=np.random.choice(train1.columns)
    print(a,b)
    plt.scatter(x=train1[a],y=train1[b],cmap='viridis')
    plt.xlabel(a)
    plt.ylabel(b)
  plot_scatter(train3)

  #Normalization - Transformation techniques
  def to_normalize(train):
    # train=train[numeric]
    func=int(input("Enter 1 for exp,2 for log, 3 for square transformation, 4 for box cox ,5 for yeo johnson: "))
    if func==1:
      train=np.exp(train)
    elif func==2:
      train=np.log(train)
    elif func==3:
      train=train**2
    elif func==4:
      pt=PowerTransformer(method='box-cox',standardize=True)
      train=pt.fit_transform(train)
    elif func==5:
      pt=PowerTransformer(method='yeo-johnson',standardize=True)
      train=pt.fit_transform(train)
    return train

  train_numeric=pd.DataFrame(to_normalize(train3.copy()))
  new_train=pd.concat([train_numeric,train4],axis=1)

  plot_histograms(train_numeric,train_numeric.columns)

  #check for correlation
  def correlation_check(train1):
    train1.corr()
    plt.figure(figsize=(20,10))
    print(sns.heatmap(train1.corr()))
    print("\n")
  correlation_check(new_train.copy())

  #outlier detection using IQR method
  #remove less for training
  import numpy as np

  def outlier_detection_iqr(train):
    l=[]
    # print(sns.boxplot(x=train[i],whis=1.5))
    #check outliers for numeric data
    for i in train.columns:
      q1=np.percentile(train[i],25,method='midpoint')
      q3=np.percentile(train[i],75,method='midpoint')
      iqr=q3-q1
      lower_lim=q1-1.5*iqr
      upper_lim=q3+1.5*iqr
      l.append([lower_lim,upper_lim])
      # print(lower_lim,upper_lim)
      # train_new=train[(train[i]>=lower_lim) & (train[i]<=upper_lim)]
      # return train
    d={}
    k=0
    for i in train.columns:
      d[i]=l[k]
      k=k+1
    print('quantile(lower and upper) limits where outliers are not detected for each column:')
    print([r for r in d.items()])
    print('\n')
    feat=random.choice(new_train.columns)
    print(sns.boxplot(x=new_train[feat],whis=1.5))

  outlier_detection_iqr(train_numeric.copy())
  return new_train

# new_train=preprocessing(train)

In [ ]:
ans=int(input('Enter 1 if you have a separate test dataset, 2 if you have only one and you want to split it: '))
ans

In [ ]:
if ans==1:
  train=t.copy()
  test=pd.read_csv(input('Enter the path for the test dataset: '))
elif ans==2:
  lim=int(input('Enter limit where you want to split: '))
  train=t.copy().iloc[0:lim]
  test=t.copy().iloc[lim:len(t)]


In [ ]:
new_train=preprocessing(train)
print(new_train)

In [ ]:
new_test=preprocessing(test)
print(new_test)

In [ ]:
def DatasetsAnalysis(new_train,new_test):
  for i in [new_train,new_test]:
    print(i)
    print(i.info())
    print(i.describe())
  sns.set(style="whitegrid")
  # new_train.reset_index()
  # new_test.reset_index()
  comparison = datacompy.Compare(new_train,new_test,join_columns=[('target')],df1_name='train',df2_name='test')
  print(comparison.report())
  l=[]
  l=list(set(new_train)&set(new_test))
  print('enter category')
  categorical=list(map(str,input().split()))
  numeric=[i for i in new_train.columns if i not in categorical]

  def plotHistCol(data,col):
    plt.subplot(1,2,1)
    sns.kdeplot(x=data[col],label='train')
    sns.kdeplot(x=data[col],label='test')
    plt.legend()

  def plotBarAndHist(new_train,new_test,numeric):
    feat=random.choice(numeric)
    # plt.subplot(1,2,1)
    # sns.histplot(x=new_train[feat],label='train',bins=100)
    # sns.histplot(x=new_test[feat],label='test',bins=100)
    # plt.legend()
    plotHistCol(new_train,feat)
    plotHistCol(new_test,feat)
    plt.figure()
    plt.subplot(1,2,1)
    sns.kdeplot(x=new_train[feat],label='train')
    sns.kdeplot(x=new_test[feat],label='test')
    plt.legend()
    plt.figure()
  print('kde plots\n')
  plotBarAndHist(new_train,new_test,numeric)

  def categoricalDist(new_train,new_test,category):
    cat=random.choice(category)
    plt.subplot(1,2,1)
    sns.countplot(x=new_train[cat])
    plt.subplot(1,2,2)
    sns.countplot(x=new_test[cat])
  categoricalDist(new_train,new_test,categorical)

  def scatterPlot(new_train,new_test,numeric):
    m=min(len(new_train),len(new_test))
    r1=random.choice(numeric)
    r2=random.choice(numeric)
    test1=new_test.iloc[0:m]
    train1=new_train.iloc[0:m]
    plt.figure()
    plt.subplot(1,2,1)
    plt.scatter(x=test1[r1],y=test1[r2],c='r')
    plt.subplot(1,2,2)
    plt.scatter(x=train1[r1],y=train1[r2],c='g')
    plt.figure()

  scatterPlot(new_train,new_test,numeric)

  def tTestStatistic(new_train,new_test,cols):
    d={}
    for feat in cols:
      train1=new_train[feat]
      test1=new_test[feat]
      #Check the means for the population distribution and sample distribution
      print('means:',train1.mean(),test1.mean())
      #for population distribution - train data
      #to check if sample data comes from the train
      #conducting a t-test for 95% confidence level
      result=stats.ttest_1samp(a=test1,popmean=train1.mean())

      # print(f'1 sample t test: {result} ')

      #checking quantiles to see if t statistic outside quantile
      #reject null hypo if true
      # for two tailed t test
      q1=stats.t.ppf(q=0.025,df=len(new_test)-1)
      q2=stats.t.ppf(q=0.975,df=len(new_test)-1)

      # print(f'quantiles : {q1} {q2}')

      #calculating p value
      t_value=result.statistic
      # For a two-tail test, the two critical values are the 2.5th and the 97.5th percentiles of the t-distribution with n−1 degrees of freedom;
      #  reject the null in favor of the alternative if the t-statistic is less than the 2.5th percentile or greater than the 97.5th percentile.

      if q1<t_value<q2:
        # print(f"t-statistic: {t_value}\n")
        #print t value and their column falling in the non rejection region,use dictionary
        d[feat]=t_value
    print([i for i in d.items()])
    print('no of cols in above:',len(d))

  print('columns which doesnt hold much difference in the mean of population distribution')
  print('printing the no of columns as well')
  tTestStatistic(new_train,new_test,new_train.columns)

  #Kolmogorov-Smirnov test-non parametric test to determine two samples come from the same distribution
  #if pvalue>alpha ->cannot reject null hypo,else reject null hypo
  def KsStatistics(new_train,new_test):
    d={}
    m=min(len(new_train),len(new_test))
    for i in new_train.columns :
      statistic,pvalue=ks_2samp(new_train[0:m][i],new_test[0:m][i])
      # print(f'test statistic {statistic}  p-value :{pvalue}')
      alpha=0.05
      if pvalue>alpha:
        d[i]=pvalue
    print([i for i in d.items()])
    print('no of cols in above:',len(d))
  print('printing cols which determine that the two samples come from the same distribution and also the number of cols')
  KsStatistics(new_train,new_test)

  def addGaussianNoise(dataset):
    #add random noise to the preprocessed data
    noise_factor=0.5
    noisyData=dataset+noise_factor*np.random.normal(loc=0.0,scale=1.0,size=dataset.shape)
    # print(noisyData)
    return pd.DataFrame(noisyData)



  #Using denoising autoencoders
  def denoisingAutoencoding(dataset,noisyData):
    input_layer = Input(shape=(dataset.shape[1],))
    encoded = Dense(64, activation='relu')(input_layer)
    decoded = Dense(dataset.shape[1], activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)

    #compiling and training the autoencoder
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(noisyData,dataset, epochs=100, batch_size=32, shuffle=True)
    return autoencoder

  # denoised_data = autoencoder.predict(noisyData)
  # return pd.DataFrame(denoised_data)

  noisyData=addGaussianNoise(new_train.copy())
  autoEncoder=denoisingAutoencoding(new_train,noisyData)

  denTrain=pd.DataFrame(autoEncoder.predict(noisyData))
  denTest=pd.DataFrame(autoEncoder.predict(new_test))
  print(denTrain)
  print('identify the categorical column from above data and enter column names:')
  cats=list(map(int,input().split()))
  nums=[x for x in denTrain.columns if x not in cats]
  denTrain[cats]=denTrain[cats].apply(lambda x:round(x))
  denTest[cats]=denTest[cats].apply(lambda x:round(x))
  print('DENOISED TRAIN DATA:',denTrain)
  plotBarAndHist(denTrain,denTest,nums)
  KsStatistics(denTrain,denTest)
  #to check are we getting similar feature importance between the two datasets?
  def featureSelection(dataset,nf):
    print('the dataset is:',dataset)
    #figure out the dataset and divide into X and y
    X=dataset.drop('target',axis=1)
    y=dataset['target']
    # X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.3,random_state=101)
    rf=RandomForestClassifier()
    rf.fit(X,y)
    importances=rf.feature_importances_
    final_train=pd.DataFrame({"Features":X.columns,"Importances":importances})
    final_train.set_index('Importances')

    #sort for good visualization
    final_train=final_train.sort_values('Importances')

    #plot the feature importance in bars
    plt.figure(figsize=(10,3))
    plt.xticks(rotation=45)
    sns.barplot(x="Features",y="Importances",data=final_train)

    #selecting features with high importance
    final_train_new=final_train.copy()
    final_train_new.sort_values('Importances',ascending=False)
    l=list(final_train_new.head(nf)["Features"])
    return l

  nf=int(input('enter limit  feature imp:-'))

  l1=featureSelection(new_train,nf)
  print('\n')
  l2=featureSelection(new_test,nf)
  print('train dataset feature importance:-',l1)
  print('train dataset feature importance:-',l2)
  # def roundingCategorical(dataset,cats):
  #   den_train1=dataset.copy()
  #   den_train1[cats]=den_train[cats].apply(lambda x:round(x))
  # return
  print(f"common importance features among the two datasets :{list(set(l1) and set(l2))}")

DatasetsAnalysis(new_train,new_test)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix
from sklearn.decomposition import PCA

In [ ]:
# 3
#  !pip install ruptures
# download it in the file youre importing

In [ ]:
import ruptures as rpt

In [ ]:
def processRegimesIdentification(new_train,new_test):
  new_train1=new_train.copy()
  new_test1=new_test.copy()
  # store and remove the target
  output1=new_train1['target']
  output2=new_test1['target']
  new_train1.drop('target',axis=1,inplace=True)
  new_test1.drop('target',axis=1,inplace=True)
  # plot a random column from your data
  def analyzeDataFeatures(new_train1):
    for i in range(2):
      plt.figure(figsize=(12,8))
      r=random.choice(new_train1.columns)

      plt.plot(new_train1.index,new_train1[r],color='g')
      plt.xlabel('Timestamp')
      plt.ylabel(f"Feature {r}")

  analyzeDataFeatures(new_train1)



  #Use changepoint analysis to separate out the control regimes
  def changePointDetection_1feature(new_train1):
    # l=[]
    d={}
    count=0
    for col in new_train1.columns:
      algo=rpt.Pelt(model='rbf').fit(new_train1[col].values)
      result=algo.predict(pen=10)
      # l.append(result)
      d[col]=result
      if count==0:
        print('Visualizing the changepoints for the first column')
        fig,ax=plt.subplots()
        ax.plot(new_train1[col],color='tab:red')
        for res in result:
          ax.axvline(x=res,color='k',linestyle='--')
        count=count+1
    # print(l)
    l1=[]
    threshold=int(input('Enter threshold to separate out the control regimes: '))
    for key in d.keys():
      if len(d[key])==threshold:
        l1.append(key)
    # print(l1)
    return l1

  controlRegimes1=changePointDetection_1feature(new_train1)

  def changePointDetection_2features(new_train1):
      print('FOR 2 FEATURES')
      r1 = random.choice(new_train1.columns)
      while True:
          r2 = random.choice(new_train1.columns)
          if r2 != r1:
              break
      print(f'Features are {r1} and {r2}')
      l=[]
      a = np.vstack([new_train1[r1].values, new_train1[r2].values])
      algo = rpt.Pelt(model='rbf').fit(a)
      result = algo.predict(pen=50)
      print(result)
      fig,ax=plt.subplots()
      ax.plot(new_train1[r1],color='orange')
      ax.plot(new_train1[r2],color='green')
      for res in result:
        ax.axvline(x=res,color='k',linestyle='--')
      if len(result)>0:
        l.append([r1,r2])
      return l
  print('CHANGEPOINT DETECTION ANALYSIS FOR 2 FEATURES')
  l=changePointDetection_2features(new_train1)
  controlRegimes=controlRegimes1

  print(controlRegimes1)
  print('No of control regimes based on a single feature: ',len(controlRegimes1))
  # controlRegimes1.append(l)
  # controlRegimes=[x for j in set(controlRegimes1)]
  conditionRegimes=list(input('Enter the condition features: ').split())
  processData=[x for x in new_train1.columns if x not in controlRegimes and x not in conditionRegimes]

  def clusteringAnalysis(X_train1,X_test1):
     def dimensionReduction(X_train1,X_test1):
      print('Reducing to 2 dimensions')
      pca=PCA(n_components=2)
      X_train=pca.fit_transform(X_train1)
      X_test=pca.transform(X_test1)
      return X_train,X_test

     X_train1,X_test1=dimensionReduction(X_train1,X_test1)

     def elbowMethodKmeans(X_train1):
      mod = KMeans(random_state=1)
      plt.figure()
      no = int(input('Enter 1 for silhouette score or 2 for calinski harabasz: '))
      d = {1: 'silhouette', 2: 'calinski_harabasz'}
      visualizer = KElbowVisualizer(mod, k=(2, 15), metric=d[no], timings=True)
      visualizer.fit(X_train1)
      visualizer.show()

     elbowMethodKmeans(X_train1)

     def visualizeClusters(labels,X):
      plt.figure()
      uLabels=np.unique(labels)
      # X['LABS']=labels
      for i in uLabels:
        indices = np.where(labels == i)[0] #get the indices
        print(f'LABEL {i}: {indices}')
        plt.scatter(X[labels == i , 0] , X[labels == i , 1] , label = i)
      plt.legend()
      plt.show()
     def KMeanClustering(X_train1):
      i = int(input('Enter the number of clusters: '))
      kmeans = KMeans(init='random', n_clusters=i, n_init=10, max_iter=300, random_state=42)
      kmeans.fit(X_train1)
      # Get the labels
      labels = kmeans.labels_
      print(f'Labels: {labels}')
      visualizeClusters(labels,X_train1)
     print('K means for train data')
     KMeanClustering(X_train1)
     print('K means for test data')
     KMeanClustering(X_test1)
  if len(controlRegimes)==0:
    print('NO CONTROL REGIMES PRESENT!')
    pass
  else:
    clusteringAnalysis(new_train1[controlRegimes],new_test1[controlRegimes])

  clusteringAnalysis(new_train1[conditionRegimes],new_test1[conditionRegimes])
  clusteringAnalysis(new_train1[processData],new_test1[processData])
  print('Clustering on the whole dataset after reducing it to 2 dimensions!')
  clusteringAnalysis(new_train1,new_test1)
processRegimesIdentification(new_train,new_test)